In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [4]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)

In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x    
net = Net()

In [8]:
# Define loss function for prediction accuracy
criterion = nn.CrossEntropyLoss()
#Define optimizer to adjust model weights to reduce loss
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [9]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[Epoch {epoch + 1}, Mini-batch {i + 1}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0  
print('Finished Training')

[Epoch 1, Mini-batch 2000] loss: 2.187
[Epoch 1, Mini-batch 4000] loss: 1.860
[Epoch 1, Mini-batch 4000] loss: 1.860
[Epoch 1, Mini-batch 6000] loss: 1.672
[Epoch 1, Mini-batch 6000] loss: 1.672
[Epoch 1, Mini-batch 8000] loss: 1.601
[Epoch 1, Mini-batch 8000] loss: 1.601
[Epoch 1, Mini-batch 10000] loss: 1.512
[Epoch 1, Mini-batch 10000] loss: 1.512
[Epoch 1, Mini-batch 12000] loss: 1.469
[Epoch 1, Mini-batch 12000] loss: 1.469
[Epoch 2, Mini-batch 2000] loss: 1.400
[Epoch 2, Mini-batch 2000] loss: 1.400
[Epoch 2, Mini-batch 4000] loss: 1.383
[Epoch 2, Mini-batch 4000] loss: 1.383
[Epoch 2, Mini-batch 6000] loss: 1.341
[Epoch 2, Mini-batch 6000] loss: 1.341
[Epoch 2, Mini-batch 8000] loss: 1.320
[Epoch 2, Mini-batch 8000] loss: 1.320
[Epoch 2, Mini-batch 10000] loss: 1.291
[Epoch 2, Mini-batch 10000] loss: 1.291
[Epoch 2, Mini-batch 12000] loss: 1.260
[Epoch 2, Mini-batch 12000] loss: 1.260
Finished Training
Finished Training


In [10]:
correct = 0
total = 0   
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f'Accuracy of the network on the 10000 test images: {100 * correct / total} %')

Accuracy of the network on the 10000 test images: 56.21 %


In [ ]:
#To save the trained model
#PATH = './cifar_net.pth'    
#torch.save(net.state_dict(), PATH)
